In [1]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
import os 
from PIL import Image
from torch.utils.data import random_split
import torch 
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import random_split
import torch 

In [2]:
trans = transforms.Compose(
    [
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ]
)

target_folder = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"
file = "/kaggle/input/celeba-dataset/list_attr_celeba.csv"
class FaceImage(Dataset):
    def __init__(self, label_file, img_dir, transform=None):
        df = pd.read_csv(label_file, delimiter=",")
        self.labels  = df[['image_id', 'Smiling']].copy()
        self.labels.replace({-1 : 0},  inplace=True)
        self.img_dir = img_dir
        self.transform = transform

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels.iloc[idx,0] ) 
        img = Image.open(img_path).convert("RGB")
        label = self.labels.iloc[idx,1]

        if self.transform:
            img = self.transform(img)
            
        return img, label 

    def __len__(self):
        return self.labels.shape[0]
    

In [3]:
data = FaceImage(file,target_folder , trans)

In [4]:
gen = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(data, [141819, 20261, 40519],
                                        generator=gen)

In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

In [6]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.6.0+cu124  Device: cuda


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.fc1 = nn.Linear(16 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 16 * 16 * 16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

In [8]:
model = CNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.BCELoss()

In [9]:
BATCH_SIZE = 128
EPOCHS = 10

In [10]:
summary(model, input_size=(32,3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 1]                   --
├─Conv2d: 1-1                            [32, 8, 64, 64]           224
├─MaxPool2d: 1-2                         [32, 8, 32, 32]           --
├─Conv2d: 1-3                            [32, 16, 32, 32]          1,168
├─MaxPool2d: 1-4                         [32, 16, 16, 16]          --
├─Linear: 1-5                            [32, 64]                  262,208
├─Linear: 1-6                            [32, 32]                  2,080
├─Linear: 1-7                            [32, 1]                   33
Total params: 265,713
Trainable params: 265,713
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 76.09
Input size (MB): 1.57
Forward/backward pass size (MB): 12.61
Params size (MB): 1.06
Estimated Total Size (MB): 15.24

In [11]:
def train(model, train_loader, optimizer, criterion, device, epoch, log_interval, batch_size):
    model.train()
    correct = 0
    train_loss = 0
    total_samples = 0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(image)[:,0]
        loss = criterion(output, label.float())
        train_loss += loss.item() * label.size(0)  # loss * batch_size로 누적
        loss.backward()
        optimizer.step()
        
        # batch별 정답수 누적
        preds = (output >= 0.5).float()
        correct += (preds == label).float().sum().item()
        total_samples += label.size(0)

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * image.size(0),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))
           
    train_loss /= total_samples
    train_accuracy = 100. * correct / total_samples

    return train_loss, train_accuracy



def evaluate(model, test_loader):
    global output, label
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output.reshape(-1), label.float()).item() * label.size(0)
            correct += ((output.reshape(-1) >= 0.5).float() == label).float().sum().item()


    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy
    
class early_stopping:
  def __init__(self, patience, verbose, delta, path='checkpoint.pt'):
    self.patience = patience
    self.verbose = verbose
    self.delta = delta
    self.count = 0
    self.best_score = None
    self.early_stop = False
    self.val_loss_min = np.inf
    self.path = path


  def __call__(self, val_loss, model):
    score = -val_loss


    if self.best_score is None:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
    elif score < self.best_score + self.delta:
      self.count += 1
      if self.verbose:
        print(f"Early Stopping counter: {self.count} out of {self.patience}")


      if self.count >= self.patience:
        self.early_stop = True  
    else:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
      self.count =0  


  def save_checkpoint(self, val_loss, model):
    if self.verbose:
      print(f"Validation loss decreased ({self.val_loss_min:.6f}) --> {val_loss:.6f}. saving model..")
    torch.save(model.state_dict(), self.path)
    self.val_loss_min = val_loss


In [ ]:
import numpy as np
early_stopping = early_stopping(patience=2, verbose=True, path='best_model.pt', delta=0)
loss_hist_train     = [0] * EPOCHS
accuracy_hist_train = [0] * EPOCHS
loss_hist_valid     = [0] * EPOCHS
accuracy_hist_valid = [0] * EPOCHS
for epoch in range(1, EPOCHS + 1):
    loss_, acc_ = train(model, train_loader, optimizer, criterion, DEVICE, epoch, log_interval = 200, batch_size=BATCH_SIZE)
    loss_hist_train[epoch-1] = loss_
    accuracy_hist_train[epoch-1] = acc_
    test_loss, test_accuracy = evaluate(model, test_loader)
    loss_hist_valid[epoch-1] = test_loss
    accuracy_hist_valid[epoch-1] = test_accuracy
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    early_stopping(test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

Train Epoch: 1 [0/141819 (0%)]	Train Loss: 0.681397
Train Epoch: 1 [6400/141819 (5%)]	Train Loss: 0.383078
Train Epoch: 1 [12800/141819 (9%)]	Train Loss: 0.277953
Train Epoch: 1 [19200/141819 (14%)]	Train Loss: 0.193259
Train Epoch: 1 [25600/141819 (18%)]	Train Loss: 0.285941
Train Epoch: 1 [32000/141819 (23%)]	Train Loss: 0.287659
Train Epoch: 1 [38400/141819 (27%)]	Train Loss: 0.307902
Train Epoch: 1 [44800/141819 (32%)]	Train Loss: 0.252554
Train Epoch: 1 [51200/141819 (36%)]	Train Loss: 0.338288
Train Epoch: 1 [57600/141819 (41%)]	Train Loss: 0.199007
Train Epoch: 1 [64000/141819 (45%)]	Train Loss: 0.275837
Train Epoch: 1 [70400/141819 (50%)]	Train Loss: 0.366775
Train Epoch: 1 [76800/141819 (54%)]	Train Loss: 0.305120
Train Epoch: 1 [83200/141819 (59%)]	Train Loss: 0.284206
